# 1. 단어 사전 기반 매핑 복원

## 1) Import 

In [48]:
import pandas as pd

## 2) Data Load

In [49]:
train = pd.read_csv('./train.csv', encoding = 'utf-8-sig')
test = pd.read_csv('./test.csv', encoding = 'utf-8-sig')

## 3) 단어 사전 생성

In [50]:
match_dict = {}

for input_text, output_text in zip(train['input'], train['output']):
    input_words = input_text.split()
    output_words = output_text.split()
    for iw, ow in zip(input_words, output_words):
        match_dict[iw] = ow  

## 4) 변환 적용

In [52]:
def replace_words(input_text, match_dict):
    words = input_text.split() 
    replaced_words = [match_dict.get(word, word) for word in words] 
    return " ".join(replaced_words)

In [53]:
converted_reviews = test['input'].apply(lambda x: replace_words(x, match_dict)).tolist()

## 5) Submission

In [55]:
submission = pd.read_csv('./sample_submission.csv', encoding = 'utf-8-sig')

In [56]:
submission['output'] = converted_reviews

In [57]:
submission.to_csv('./baseline_submission.csv', index = False, encoding = 'utf-8-sig')

# 2. LLM 활용 (Gemma)

## 1) Import 

In [ ]:
import pandas as pd 
import torch 
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig, pipeline

## 2) Data Load

In [3]:
train = pd.read_csv('./train.csv', encoding = 'utf-8-sig')
test = pd.read_csv('./test.csv', encoding = 'utf-8-sig')

In [ ]:
samples = []

for i in range(10):
    sample = f"input : {train['input'][i]} \n output : {train['output'][i]}"
    samples.append(sample)

## 3) Model Load

In [ ]:
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type= 'nf4',
    bnb_4bit_use_double_quant = True,
    bnb_4bit_compute_dtype=torch.bfloat16
)
model_id = 'beomi/gemma-ko-7b'
model = AutoModelForCausalLM.from_pretrained(model_id, quantization_config = bnb_config, device_map={"":0})
tokenizer = AutoTokenizer.from_pretrained(model_id)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = 'right'

## 4) Inference

In [ ]:
pipe = pipeline(
    task="text-generation",
    model=model, 
    tokenizer=tokenizer 
)

restored_reviews = []


for index, row in test.iterrows():
    query = row['input']  
    
    messages = [
        {
            "role": "system",
            "content": (
                "You are a helpful assistant specializing in restoring obfuscated Korean reviews. "
                "Your task is to transform the given obfuscated Korean review into a clear, correct, "
                "and natural-sounding Korean review that reflects its original meaning. "
                "Below are examples of obfuscated Korean reviews and their restored forms:\n\n"
                f"Example, {samples}"  
                "Spacing and word length in the output must be restored to the same as in the input. "
                "Do not provide any description. Print only in Korean."
            )
        },
        {
            "role": "user",
            "content": f"input : {query}, output : "
        },
    ]
    
    prompt = "\n".join([m["content"] for m in messages]).strip()
    

    outputs = pipe(
        prompt,
        do_sample=True,
        temperature=0.2,
        top_p=0.9,
        max_new_tokens=len(query),
        eos_token_id=pipe.tokenizer.eos_token_id
    )
    
    generated_text = outputs[0]['generated_text']
    result = generated_text[len(prompt):].strip()
        

    restored_reviews.append(result)

## 5) Submission

In [23]:
submission = pd.read_csv('./sample_submission.csv', encoding = 'utf-8-sig')

In [ ]:
submission['output'] = restored_reviews

In [ ]:
submission.to_csv('./baseline_submission.csv', index = False, encoding = 'utf-8-sig')